# Traiter des données tabulaires avec Pandas

L'analyse statistique a généralement pour base des **données tabulaires**, dans lesquelles chaque ligne représente une observation et chaque colonne une variable. Pour traiter ce type de données et y appliquer facilement les méthodes d'analyse de données standards, des objets dédiés ont été conçus : les `DataFrames`. Les utilisateurs de `R` connaissent bien cette structure de données, qui est native à ce langage orienté statistique. En `Python`, langage généraliste, cet objet n'existe pas nativement. Heureusement, une librairie très complete et bien pratique, pensée comme une surcouche à `NumPy`, introduit en `Python` l'objet `DataFrame` et permet la manipulation et l'analyse de données de manière simple et intuitive : `Pandas`.

On commence par importer la librairie `Pandas`. L'usage est courant est de lui attribuer l'alias `pd` afin de simplifier les futurs appels aux objets et fonctions du package. On importe également `NumPy` car on va comparer les objets fondamentaux des deux packages.

In [2]:
import pandas as pd
import numpy as np

## Structures de données

Pour bien comprendre le fonctionnement de `Pandas`, il faut s'intéresser à ses objets fondamentaux. On va donc d'abord étudier les `Series`, dont la concaténation permet de construire un `DataFrame`. 

### La `Series`

Une Series est un conteneur de données unidimensionnel pouvant accueillir n'importe quel type de données (entiers, *strings*, objets Python...). Une Series est néanmoins d'un type donné : une Series ne contenant que des entiers sera de type `int`, et une Series contenant des objets de différente nature sera de type `object`. Construisons notre première Series à partir d'une liste pour vérifier ce comportement.

In [3]:
l = [1, "X", 3]
s = pd.Series(l)
print(s)

0    1
1    X
2    3
dtype: object


On peut notamment accéder aux données d'une Series par position, comme pour une liste ou un array.

In [4]:
print(s[1])

X


A priori, on ne voit pas beaucoup de différence entre une Series et un *array* `NumPy` à 1 dimension. Pourtant, il existe une différence de taille qui est la présence d'un index : les observations ont un label associé. Lorsqu'on crée une Series sans rien spécifier, l'index est automatiquement fixé aux entiers de 0 à n (le nombre d'éléments de la Series). Mais il est possible de passer un index spécifique (ex : des dates, des noms de communes, etc.).

In [5]:
s = pd.Series(l, index=["a", "b", "c"])
print(s)

a    1
b    X
c    3
dtype: object


Ce qui permet d'accéder aux données par label :

In [6]:
s["b"]

'X'

Cette différence apparaît secondaire à première vue, mais deviendra essentielle pour la construction du DataFrame. Pour le reste, les Series se comportent de manière très proche des arrays NumPy : les calculs sont vectorisés, on peut directement faire la somme de deux Series, etc. D'ailleurs, on peut très facilement convertir une Series en array via l'attribut `values`. Ce qui, naturellement, fait perdre l'index...

In [7]:
s = pd.Series(l, index=["a", "b", "c"])
s.values

array([1, 'X', 3], dtype=object)

### Le `DataFrame`

Fondamentalement, un DataFrame consiste en une collection de Series, alignées par les index. Cette concaténation construit donc une table de données, dont les Series correspondent aux colonnes, et dont l'index identifie les lignes. La figure suivante ([source](https://medium.com/epfl-extension-school/selecting-data-from-a-pandas-dataframe-53917dc39953)) permet de bien comprendre cette structure de données.

<div>
<img src="img/structure_df.png" width="800">
</div>

Un DataFrame peut être construit de multiples manières. En pratique, on construit généralement un DataFrame directement à partir de fichiers de données tabulaires (ex : CSV, excel), rarement à la main. On illustrera donc seulement la méthode de construction manuelle la plus usuelle : à partir d'un dictionnaire de données.

In [152]:
df = pd.DataFrame(
    data = {
        "var1": 1.0,
        "var2": np.random.randint(-10, 10, 6),
        "experiment": ["test", "train", "test", "train", "train", "train"],
        "date": ["2022-01-01", "2022-01-02", "2022-01-03", "2022-01-04", "2022-01-05", "2022-01-06"],
        "sample": ["sample1", "sample2", "sample1", "sample1", "sample2", "sample3"]
    }
)

df

,var1,var2,experiment,date,sample
0,1.0,-5,test,2022-01-01,sample1
1,1.0,9,train,2022-01-02,sample2
2,1.0,-3,test,2022-01-03,sample1
3,1.0,8,train,2022-01-04,sample1
4,1.0,-10,train,2022-01-05,sample2
5,1.0,-5,train,2022-01-06,sample3


Un DataFrame Pandas dispose d'un ensemble d'attributs utiles que nous allons découvrir tout au long de ce tutoriel. Pour l'instant, intéressons nous aux plus basiques : l'index et le nom des colonnes. Par défaut, l'index est initialisé comme pour les Series à la liste des positions des observations. On aurait pu spécifier un index alternatif lors de la construction du DataFrame en spécifiant l'argument `index` de la fonction `pd.DataFrame`.

In [153]:
df.index

RangeIndex(start=0, stop=6, step=1)

In [154]:
df.columns

Index(['var1', 'var2', 'experiment', 'date', 'sample'], dtype='object')

Souvent, plutôt que de spécifier un index à la main lors de la construction du DataFrame, on va vouloir utiliser une certaine colonne du DataFrame comme index. On utilise pour cela la méthode `set_index` associée aux DataFrames.

In [155]:
df = df.set_index("date")
df

,var1,var2,experiment,sample
date,,,,
2022-01-01,1.0,-5,test,sample1
2022-01-02,1.0,9,train,sample2
2022-01-03,1.0,-3,test,sample1
2022-01-04,1.0,8,train,sample1
2022-01-05,1.0,-10,train,sample2
2022-01-06,1.0,-5,train,sample3


L'attribut index a naturellement changé :

In [156]:
df.index

Index(['2022-01-01', '2022-01-02', '2022-01-03', '2022-01-04', '2022-01-05',
       '2022-01-06'],
      dtype='object', name='date')

## Sélectionner des données

Lorsqu'on analyse des données, la première étape consiste souvent à limiter son échantillon, que ce soit en termes d'observations ou de variables utilisées. Les DataFrames (et les Series) Pandas disposent d'un ensemble de méthodes bien utiles pour cela.

### Sélectionner des colonnes

Pour sélectionner des colonnes, la manière la plus simple est d'utiliser la méthode `[]`, familière aux utilisateurs de `R`. Si l'on sélectionne une seule colonne, l'objet retourné, unidimensionnel, est une Series.

In [157]:
exp = df["experiment"]
exp

date
2022-01-01     test
2022-01-02    train
2022-01-03     test
2022-01-04    train
2022-01-05    train
2022-01-06    train
Name: experiment, dtype: object

In [158]:
type(exp)

pandas.core.series.Series

Comme les arrays NumPy, les Series et les DataFrames possèdent l'attribut `.shape`, qui donne à la fois le nombre de dimensions de l'objet et le nombre d'éléments par dimension.

In [159]:
exp.shape

(6,)

Il est également possible d'accéder aux éléments d'une colonne via la syntaxe `df.nom_colonne`. Mais cette approche peut parfois causer des erreurs, notamment dans le cas de variables contenant des noms d'espace. Il est donc préférable de se limiter à la méthode `[]` pour sélectionner des colonnes, plus explicite.

In [160]:
df.experiment

date
2022-01-01     test
2022-01-02    train
2022-01-03     test
2022-01-04    train
2022-01-05    train
2022-01-06    train
Name: experiment, dtype: object

On peut bien entendu sélectionner plusieurs colonnes, en passant une liste de noms de colonnes. L'objet retourné, bidimensionnel, est alors un DataFrame.

In [161]:
subdf = df[["experiment", "var1", "var2"]]
subdf

,experiment,var1,var2
date,,,
2022-01-01,test,1.0,-5
2022-01-02,train,1.0,9
2022-01-03,test,1.0,-3
2022-01-04,train,1.0,8
2022-01-05,train,1.0,-10
2022-01-06,train,1.0,-5


In [162]:
type(subdf)

pandas.core.frame.DataFrame

In [163]:
subdf.shape

(6, 3)

### Sélectionner des lignes

Dès lors que l'on souhaite sélectionner des lignes (ou des lignes ET des colonnes), on va devoir utiliser des méthodes spécifiques.

La méthode `.iloc[]` permet de sélectionner des lignes par positions, quelle que soit la nature de l'index.

In [164]:
df.iloc[1]

var1              1.0
var2                9
experiment      train
sample        sample2
Name: 2022-01-02, dtype: object

In [165]:
df.iloc[1:4]

,var1,var2,experiment,sample
date,,,,
2022-01-02,1.0,9,train,sample2
2022-01-03,1.0,-3,test,sample1
2022-01-04,1.0,8,train,sample1


En pratique, sélectionner des données par position limite la reproductibilité : si les données en entrée changent, le code produit n'est plus valide. On préférera donc sélectionner les données par labels. Pour cela, on utilise la méthode `.loc[]`.

In [166]:
df.loc["2022-01-02"]

var1              1.0
var2                9
experiment      train
sample        sample2
Name: 2022-01-02, dtype: object

In [167]:
df.loc[["2022-01-02", "2022-01-03", "2022-01-04"]]

,var1,var2,experiment,sample
date,,,,
2022-01-02,1.0,9,train,sample2
2022-01-03,1.0,-3,test,sample1
2022-01-04,1.0,8,train,sample1


On peut utiliser le principe du `slicing` pour récupérer toutes les observations comprises entre deux labels donnés.

In [168]:
df.loc["2022-01-02":"2022-01-05"]

,var1,var2,experiment,sample
date,,,,
2022-01-02,1.0,9,train,sample2
2022-01-03,1.0,-3,test,sample1
2022-01-04,1.0,8,train,sample1
2022-01-05,1.0,-10,train,sample2


En utilisant la méthode `.loc[]` (et `.iloc[]`), il est possible de sélectionner à la fois des lignes et des colonnes. La syntaxe est la suivante : `df.loc[labels_lignes, labels_colonnes]`.

In [169]:
df.loc[["2022-01-02", "2022-01-03", "2022-01-04"], ["var1", "var2"]]

,var1,var2
date,,
2022-01-02,1.0,9
2022-01-03,1.0,-3
2022-01-04,1.0,8


En spécifiant un label donné pour les lignes et les colonnes, on peut récupérer directement une valeur scalaire du DataFrame.

In [170]:
df.loc["2022-01-03", "var2"]

-3

### Sélectionner les données selon des conditions

Souvent, on souhaite filtrer nos données selon des conditions logiques (condition d'âge, département de résidence, etc.). Pandas utilise les principes de masques booléens et de vectorisation des opérations que l'on a vu avec NumPy. Filtrer un DataFrame selon des conditions logiques est donc très simple.

In [171]:
df["experiment"] == "train"

date
2022-01-01    False
2022-01-02     True
2022-01-03    False
2022-01-04     True
2022-01-05     True
2022-01-06     True
Name: experiment, dtype: bool

La condition logique renvoie une série de booléens, que l'on peut donc utiliser comme masque pour filtrer un DataFrame.

In [172]:
df[df["experiment"] == "train"]

,var1,var2,experiment,sample
date,,,,
2022-01-02,1.0,9,train,sample2
2022-01-04,1.0,8,train,sample1
2022-01-05,1.0,-10,train,sample2
2022-01-06,1.0,-5,train,sample3


En utilisant `.loc[]`, on peut à la fois filtrer les observations selon une condition et ne retenir que certaines variables.

In [173]:
df.loc[df["experiment"] == "train", "var2"]

date
2022-01-02     9
2022-01-04     8
2022-01-05   -10
2022-01-06    -5
Name: var2, dtype: int64

Il est bien entendu possible de filtrer les données en utilisant plusieurs conditions, combinées via un opérateur logique. Par exemple, l'opérateur ET, symbolisé par `&`. Attention, en Pandas, il est essentiel de mettre des parenthèses autour de chaque condition logique que l'on combine.

In [174]:
df.loc[(df["experiment"] == "train") & (df.index >= "2022-01-04")]

,var1,var2,experiment,sample
date,,,,
2022-01-04,1.0,8,train,sample1
2022-01-05,1.0,-10,train,sample2
2022-01-06,1.0,-5,train,sample3


De même, on peut utiliser l'opérateur OU (inclusif), symbolisé par `|`.

In [175]:
df.loc[(df["experiment"] == "train") | (df.index >= "2022-01-04")]

,var1,var2,experiment,sample
date,,,,
2022-01-02,1.0,9,train,sample2
2022-01-04,1.0,8,train,sample1
2022-01-05,1.0,-10,train,sample2
2022-01-06,1.0,-5,train,sample3


On remarque au passage une propriété intéressante : Pandas a automatiquement détecté que les *strings* de la colonne `date` étaient d'un type particulier (format d'une date), et nous laisse donc faire des opérations de comparaisons pour filtrer par date. Pratique !

In [176]:
df.index >= '2022-01-03'

array([False, False,  True,  True,  True,  True])

Enfin, la méthode `.isin()` est très utilisée pour le fitrage de données catégorielles. Elle permet de ne retenir que les observations qui présentent certaines modalités d'une ou de plusieurs variables.

In [179]:
df["sample"].isin(["sample2", "sample3"])

date
2022-01-01    False
2022-01-02     True
2022-01-03    False
2022-01-04    False
2022-01-05     True
2022-01-06     True
Name: sample, dtype: bool

In [180]:
df[df["sample"].isin(["sample2", "sample3"])]

,var1,var2,experiment,sample
date,,,,
2022-01-02,1.0,9,train,sample2
2022-01-05,1.0,-10,train,sample2
2022-01-06,1.0,-5,train,sample3


## Explorer des données tabulaires

### Importer des données

### Visualiser un échantillon des données

### Obtenir une vue d'ensemble des données

### Calculer des statistiques descriptives

## Principales manipulations de données

### Transformer les données

#### Transformer un DataFrame

#### Transformer les colonnes

#### Transformer les lignes

### Trier les valeurs

### Traiter les données textuelles

### Traiter les valeurs manquantes

### Opérations par groupes

### Joindre des tables

#### Concaténer des tables

#### Fusionner des tables